In [1]:
import pandas as pd
import numpy as np
import random
import pymysql
import datetime

## 데이터 가져오기

In [2]:
# Mysql에서 데이터 가져오는 함수
def get_mysql_data(str):
    conn = pymysql.connect(host = "localhost",
             user= "root", password = "1234", charset = "utf8")
    cursor = conn.cursor()
    cursor.execute(str)
    data = cursor.fetchall()
    return data

In [3]:
menu_table = pd.DataFrame(columns = ['menu_id','menu_name','restaurant_id','url'], data = get_mysql_data('SELECT * FROM young_cheline.menu'))
menu_table.head()
len(menu_table)

35

In [4]:
user_table = pd.DataFrame(columns = ['id','active','email','password','role','temp_pw','time','user_name'], data = get_mysql_data('SELECT * FROM young_cheline.user'))
user_table = user_table[user_table['user_name'].notnull()]

## 데이터 전처리

In [5]:
# 부여된 가중치를 활용하여 표본을 n번 추출
def Random_Weighted_Sampling(menu_df,user_df,n):
    prs_data = []
    for i in range(len(menu_df)):
        prs_data.append(menu_df.iloc[i,:].tolist() + random.choices(range(9,15), k=2) + random.choices(range(5,11), k=2) + random.choices(range(1,7), k=1))
    df = pd.DataFrame(columns = ['menu_id','menu_name','restaurant_id','url','1','2','3','4','5'], data = prs_data)
    df_n = len(df)
    user_n = len(user_df)
    id_data = [0 for i in range(user_n)]
    user_df['count']=id_data
    rws_data = []
    #db연결
    conn = pymysql.connect(host = "localhost",
    user= "root", password = "1234", charset = "utf8")
    cursor = conn.cursor()
    #계산
    for i in range(n):
        # 무작위로 메뉴 선택
        random_index = random.randrange(df_n)
        user_index = random.randrange(user_n)
        # 메뉴의 정보
        menu_id = df.iloc[random_index].loc['menu_id'].tolist()
        time = datetime.datetime.now()
        user_name = user_df.iloc[user_index].loc['user_name']
        number = i+1
        restaurant_id = df.iloc[random_index].loc['restaurant_id']
        url = df.iloc[random_index].loc['url']
        # 메뉴의 가중치
        weight2 = df.iloc[random_index,[4,8]].to_numpy()
        weightT = df.iloc[random_index,[4,5,6,8]].to_numpy()
        weight4 = df.iloc[random_index,[4,6,7,8]].to_numpy()
        weight5 = df.iloc[random_index,[4,5,6,7,8]].to_numpy()
        # 가중치에 따른 확률 계산
        percent2 = list(weight2/np.sum(weight2))
        percentT = list(weightT/np.sum(weightT))
        percent4 = list(weight4/np.sum(weight4))
        percent5 = list(weight5/np.sum(weight5))
        # 확률에 맞게 추출
        flavor = np.random.choice([1,2,3,4],p=percentT)
        cleaning = np.random.choice([None,1,2,3],p=percent4)
        service = np.random.choice([None,1,2,3],p=percent4)
        plating = np.random.choice([None,1,2,3],p=percent4)
        price = np.random.choice([None,1,2,3,4],p=percent5)        
        couple = np.random.choice([None,0],p=percent2)        
        family = np.random.choice([None,1],p=percent2)        
        solo = np.random.choice([None,2],p=percent2)        
        drink = np.random.choice([None,3],p=percent2)        
        friend = np.random.choice([None,4],p=percent2)        
        ##유저가 평가한 횟수 카운팅
        user_df.at[user_index,'count'] = user_df.iloc[user_index].loc['count'].tolist()+1
        id = user_df.iloc[user_index].loc['count']
        
        #db insert
        cursor.execute('''INSERT INTO young_cheline.evaluate (menu_id, time, user_name, cleaning, couple, drink, family, friend, number, 
                plating, price, restaurant_id, service, solo, flavor, url, id) VALUE(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'''
              ,(menu_id,time,user_name,cleaning,couple,drink,family,friend,number,plating,price,restaurant_id,service,solo,flavor,url,id))
        conn.commit()
        
        # 모두 엮어서 하나의 데이터프레임으로 만들기
        rws_data.append([menu_id,time,user_name,cleaning,couple,drink,family,friend,number,plating,price,restaurant_id,service,solo,flavor,url,id])
        
    result_df = pd.DataFrame(columns = ['menu_id','time','user_name','cleaning','couple','drink','family','friend','number','plating','price','restaurant_id','service','solo','flavor','url','id'],data = rws_data)
    return result_df

In [6]:
rwsdf = Random_Weighted_Sampling(menu_table,user_table,100000)

In [7]:
rwsdf[rwsdf['user_name']=='ehddbs4521']

,menu_id,time,user_name,cleaning,couple,drink,family,friend,number,plating,price,restaurant_id,service,solo,flavor,url,id
91,2,2023-12-07 01:08:10.779504,ehddbs4521,NaN,NaN,3.0,NaN,4.0,92,3.0,3.0,15752650,3.0,2.0,2,https://youngcheline.s3.ap-northeast-2.amazona...,1
154,8,2023-12-07 01:08:11.016268,ehddbs4521,NaN,NaN,NaN,NaN,4.0,155,2.0,NaN,10306127,1.0,2.0,3,https://youngcheline.s3.ap-northeast-2.amazona...,2
2357,12,2023-12-07 01:08:18.935901,ehddbs4521,1.0,0.0,NaN,1.0,NaN,2358,2.0,2.0,27407049,1.0,NaN,1,https://youngcheline.s3.ap-northeast-2.amazona...,3
11893,4,2023-12-07 01:08:53.856983,ehddbs4521,NaN,NaN,NaN,NaN,NaN,11894,NaN,NaN,11385265,1.0,NaN,2,https://youngcheline.s3.ap-northeast-2.amazona...,4
19698,34,2023-12-07 01:09:22.739149,ehddbs4521,2.0,0.0,NaN,NaN,4.0,19699,1.0,4.0,86660948,1.0,NaN,4,https://youngcheline.s3.ap-northeast-2.amazona...,5
31664,27,2023-12-07 01:10:06.333467,ehddbs4521,NaN,0.0,NaN,1.0,NaN,31665,NaN,1.0,26353366,NaN,NaN,1,https://youngcheline.s3.ap-northeast-2.amazona...,6
42542,34,2023-12-07 01:10:46.170835,ehddbs4521,NaN,NaN,NaN,1.0,NaN,42543,2.0,2.0,86660948,NaN,NaN,1,https://youngcheline.s3.ap-northeast-2.amazona...,7
45700,34,2023-12-07 01:10:57.631612,ehddbs4521,NaN,0.0,3.0,NaN,NaN,45701,1.0,1.0,86660948,3.0,NaN,2,https://youngcheline.s3.ap-northeast-2.amazona...,8
55070,34,2023-12-07 01:11:32.201390,ehddbs4521,NaN,NaN,3.0,NaN,4.0,55071,NaN,NaN,86660948,2.0,NaN,2,https://youngcheline.s3.ap-northeast-2.amazona...,9
64126,28,2023-12-07 01:12:05.096282,ehddbs4521,NaN,NaN,NaN,NaN,NaN,64127,NaN,NaN,1956466361,NaN,NaN,4,https://youngcheline.s3.ap-northeast-2.amazona...,10


In [184]:
menu_table
user_table

menu_df = menu_table
user_df = user_table

In [185]:
prs_data = []
for i in range(len(menu_df)):
    prs_data.append(menu_df.iloc[i,:].tolist() + random.choices(range(9,15), k=2) + random.choices(range(5,11), k=2) + random.choices(range(1,7), k=1))
df = pd.DataFrame(columns = ['menu_id','menu_name','restaurant_id','url','1','2','3','4','5'], data = prs_data)

In [186]:
df_n = len(df)
user_n = len(user_df)
id_data = [0 for i in range(user_n)]
user_df['count']=id_data
rws_data = []

In [187]:
#db연결
conn = pymysql.connect(host = "localhost",
user= "root", password = "1234", charset = "utf8")
cursor = conn.cursor()

In [203]:
random_index = random.randrange(df_n)
user_index = random.randrange(user_n)
# 메뉴의 정보
menu_id = df.iloc[random_index].loc['menu_id'].tolist()
time = datetime.datetime.now()
user_name = user_df.iloc[user_index].loc['user_name']
number = i+1
restaurant_id = df.iloc[random_index].loc['restaurant_id']
url = df.iloc[random_index].loc['url']
# 메뉴의 가중치
weight2 = df.iloc[random_index,[4,8]].to_numpy()
weightT = df.iloc[random_index,[4,5,6,8]].to_numpy()
weight4 = df.iloc[random_index,[4,6,7,8]].to_numpy()
weight5 = df.iloc[random_index,[4,5,6,7,8]].to_numpy()
# 가중치에 따른 확률 계산
percent2 = list(weight2/np.sum(weight2))
percentT = list(weightT/np.sum(weightT))
percent4 = list(weight4/np.sum(weight4))
percent5 = list(weight5/np.sum(weight5))
# 확률에 맞게 추출
flavor = np.random.choice([1,2,3,4],p=percentT)
cleaning = np.random.choice([None,1,2,3],p=percent4)
service = np.random.choice([None,1,2,3],p=percent4)
plating = np.random.choice([None,1,2,3],p=percent4)
price = np.random.choice([None,1,2,3,4],p=percent5)        
couple = np.random.choice([None,0],p=percent2)        
family = np.random.choice([None,1],p=percent2)        
solo = np.random.choice([None,2],p=percent2)        
drink = np.random.choice([None,3],p=percent2)        
friend = np.random.choice([None,4],p=percent2)        
##유저가 평가한 횟수 카운팅
user_df.at[user_index,'count'] = user_df.iloc[user_index].loc['count'].tolist()+1
id = user_df.iloc[user_index].loc['count'].tolist()

In [204]:
type(id)

int

In [212]:
# db insert
sql = '''INSERT INTO young_cheline.evaluate (menu_id, time, user_name, cleaning, couple, drink, family, friend, number, 
plating, price, restaurant_id, service, solo, flavor, url, id) VALUE({},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{});'''.format(menu_id,time,user_name,cleaning,couple,drink,family,friend,number,plating,price,restaurant_id,service,solo,flavor,url,id)

In [213]:
sql

'INSERT INTO young_cheline.evaluate (menu_id, time, user_name, cleaning, couple, drink, family, friend, number, \nplating, price, restaurant_id, service, solo, flavor, url, id) VALUE(33,2023-12-06 21:56:47.465680,fb108054-42,None,None,None,None,None,35,2,2,2084958402,1,None,3,https://youngcheline.s3.ap-northeast-2.amazonaws.com/2020305821_%EC%B9%98%EC%A6%88%EB%8F%88%EA%B9%8C%EC%8A%A4_%E1%84%83%E1%85%A1%E1%84%8B%E1%85%AE%E1%86%AB%E1%84%85%E1%85%A9%E1%84%83%E1%85%B3.jpeg,1);'

In [216]:
cursor.execute('''INSERT INTO young_cheline.evaluate (menu_id, time, user_name, cleaning, couple, drink, family, friend, number, 
                plating, price, restaurant_id, service, solo, flavor, url, id) VALUE(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'''
              ,(menu_id,time,user_name,cleaning,couple,drink,family,friend,number,plating,price,restaurant_id,service,solo,flavor,url,id))

1

In [ ]:
print('data insert to db')

In [198]:
print(friend,type(friend))

4 <class 'int'>


In [79]:
# db에 데이터 넣기
def insert_evaluate_data(df):
    conn = pymysql.connect(host = "localhost",
             user= "root", password = "1234", charset = "utf8")
    cursor = conn.cursor()
    for i in range(len(df)):
        data = df.iloc[i].tolist()
        sql = "INSERT INTO evaluate (menu_id, time, user_name, cleaning, couple, drink, family, friend, number, plating, price, restaurant_id, service, solo, taste, url, id) VALUE({},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{})".format(data[0],data[1],data[2],data[3],data[4],data[5],data[6],data[7],data[8],data[9],data[10],data[11],data[12],data[13],data[14],data[15],data[16])
        cursor.excute(sql)
        db.commit()
    return print('DB Insert Fin')